# How to access LUMI's Extremes Digital Twin data using earthkit and the Polytope API

This document serves as a guide for utilizing the [earthkit](https://earthkit.readthedocs.io/en/latest/) library to extract DestinE data stored in [LUMI](https://www.lumi-supercomputer.eu/lumi-selected-as-a-platform-for-destination-earths-climate-change-adaptation-digital-twin/)  and subsequently visualize it using plots usinc [xdggs](https://xdggs.readthedocs.io/en/latest/) software. Earthkit functions as a set of tools specifically designed for working with geospatial data, while LUMI represents a data storage and computation facility. DestinE data will be retrieved from LUMI through the earthkit library's functionalities. Following the data extraction, the document will provide instructions on how to generate plots to visually represent the DestinE data.  We download the high resolution Climate digital twin data based on ICON model. We download 3D ocean potential temperatre data.

[Polytope](https://polytope.ecmwf.int/openapi/), an API offered by the European Centre for Medium-Range Weather Forecasts (ECMWF) is leveraged through earthkit to achieve this goal. 

## What you will learn
* How to access and preview the dataset
* How to select the data
* How to plot the results

## Prerequisites
### DestinE Platform Credentials

You need to have an account on the [Destination Earth Platform](https://auth.destine.eu/realms/desp/account).

#### ⚠️ Warning: Authorized Access Only
The usage of this notebook and data access is reserved only to authorized user groups.

On code.insula.destin.eu, before executing the code, one need to create local eviroment ; 

```
curl -L https://micro.mamba.pm/install.sh | bash
source ~/.bashrc
micromamba create -n pangeo -c conda-forge python==3.12 pangeo-notebook pint-xarray cf_xarray -y 
micromamba activate pangeo
pip install xdggs 
pip install earthkit
python -m ipykernel install --user --name=pangeo --display-name "Python (pangeo)"
```


In [1]:
import math
import cf_xarray.units
import earthkit.data
import numpy as np
import pint_xarray
import xarray as xr
import xdggs

#### ⚠️ To change:  Download the time series at once, (the size which is good for chunk size).
polytope command for group download 


In [6]:


#ICON
request = {
    "class": "d1",
    "dataset": "climate-dt",
    "activity": "ScenarioMIP",
    "experiment": "SSP3-7.0",
    "model": "ICON",
    "generation": "1",
    "realization": "1",
    "resolution": "high",
    "expver": "0001",
    "stream": "clte",
#    "time": "0000",
   "time": "0000/0100/0200/0300/0400/0500/0600/0700/0800/0900/1000/1100/1200/1300/1400/1500/1600/1700/1800/1900/2000/2100/2200/2300",
    "type": "fc",
    "levtype": "o3d",
    "levelist": "1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50/51/52/53/54/55/56/57/58/59/60/61/62/63/64/65/66/67/68/69/70/71/72/73/74/75",
    "param": "263501"
}



In [7]:
# data is an earthkit streaming object but with stream=False will download data immediately
data = earthkit.data.from_source(
    "polytope",
    "destination-earth",
    request,
    address="polytope.lumi.apps.dte.destination-earth.eu",
    stream=False,
)


2024-12-02 02:46:05 - INFO - Key read from /home/jovyan/.polytopeapirc
2024-12-02 02:46:05 - INFO - Sending request...
{'request': 'activity: ScenarioMIP\n'
            'class: d1\n'
            'dataset: climate-dt\n'
            'experiment: SSP3-7.0\n'
            "expver: '0001'\n"
            "generation: '1'\n"
            'levelist: '
            '1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50/51/52/53/54/55/56/57/58/59/60/61/62/63/64/65/66/67/68/69/70/71/72/73/74/75\n'
            'levtype: o3d\n'
            'model: ICON\n'
            "param: '263501'\n"
            "realization: '1'\n"
            'resolution: high\n'
            'stream: clte\n'
            'time: '
            '0000/0100/0200/0300/0400/0500/0600/0700/0800/0900/1000/1100/1200/1300/1400/1500/1600/1700/1800/1900/2000/2100/2200/2300\n'
            'type: fc\n',
 'verb': 'retrieve'}
2024-12-02 02:46:05 - INFO - Polytop

f2e570b5-3f11-4d45-ac0c-bcb6ec18a15e.grib:   0%|          | 0.00/629M [00:00<?, ?B/s]

In [8]:
data.describe()

,,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
shortName,typeOfLevel,,,,,,,,,,
avg_thetao,oceanModelLayer,"1,2,...",20241202,0,24,None,263501,d1,clte,fc,0001


In [11]:
###Time updated but when transforming to xarray, i see only one time step! 
### if thie really does not work, may be we should use pangeo-forge recepies and create zarr file (chunked in time) and re-chunk it to have bigger time chunk later on?
###
data.to_xarray_cfgrib(user_kwargs={})

<xarray.Dataset> Size: 4GB
Dimensions:          (time: 1, step: 1, oceanModelLayer: 72, values: 12582912)
Coordinates:
  * time             (time) datetime64[ns] 8B 2024-12-02
  * step             (step) timedelta64[ns] 8B 1 days
  * oceanModelLayer  (oceanModelLayer) float64 576B 1.0 2.0 3.0 ... 71.0 72.0
    latitude         (values) float64 101MB ...
    longitude        (values) float64 101MB ...
    valid_time       (time, step) datetime64[ns] 8B ...
Dimensions without coordinates: values
Data variables:
    avg_thetao       (time, step, oceanModelLayer, values) float32 4GB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          1003
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-02T02:47 GRIB to CDM+CF via cfgrib-0.9.1...

In [10]:
ds = data.to_xarray_cfgrib(user_kwargs={})
x = (ds["values"].size) / 12
level = math.log(x, 4)
level = int(level)
chunk_level=2

print('healpix level is',level,'we chunk for each healpix in chunk_level=',chunk_level)

ds = (
    ds.rename_dims({"values": "cell_id"})
    .assign_coords(
        cell_ids=(
            "cell_id",
            np.arange(12 * 4**level),
            {"grid_name": "healpix", "level": level, "indexing_scheme": "nested"},
        )
    )
)
ds=ds.chunk(cell_id=(ds.cell_id.size) / (12 * 4**chunk_level))
ds


healpix level is 10 we chunk for each healpix in chunk_level= 2


<xarray.Dataset> Size: 4GB
Dimensions:          (time: 1, step: 1, oceanModelLayer: 72, cell_id: 12582912)
Coordinates:
  * time             (time) datetime64[ns] 8B 2024-12-02
  * step             (step) timedelta64[ns] 8B 1 days
  * oceanModelLayer  (oceanModelLayer) float64 576B 1.0 2.0 3.0 ... 71.0 72.0
    latitude         (cell_id) float64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
    longitude        (cell_id) float64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
    valid_time       (time, step) datetime64[ns] 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
    cell_ids         (cell_id) int64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
Dimensions without coordinates: cell_id
Data variables:
    avg_thetao       (time, step, oceanModelLayer, cell_id) float32 4GB dask.array<chunksize=(1, 1, 72, 65536), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          1003
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-02T02:46 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
import numpy as np
# see https://confluence.ecmwf.int/pages/viewpage.action?pageId=414268596#Technicaldetails(ClimateDT)-ICON
# Depth data for the 72 levels
depth_top = np.array([
    0, 2, 4.2, 6.7, 9.5, 12.6, 16.1, 20, 24.4, 29.3, 34.7, 40.6, 47, 54.1, 61.8,
    70.2, 79.4, 89.5, 100.5, 112.5, 125.7, 140.1, 155.8, 172.9, 191.6, 212, 234.3,
    258.6, 285.1, 314, 345.5, 379.8, 417.1, 457.7, 500.8, 546.1, 592.9, 641.3,
    691.3, 743, 796.4, 851.6, 908.6, 967.5, 1028.3, 1091.2, 1157.8, 1230.4, 1311,
    1401.6, 1501.8, 1611.8, 1732.1, 1860.8, 1998.2, 2144.6, 2300.3, 2465.5, 2640.3,
    2824.7, 3018.8, 3222.4, 3435.3, 3657.2, 3887.7, 4126.2, 4372.1, 4624.5, 4882.6,
    5145.4, 5411.8, 5680.7
])

depth_bottom = np.array([
    2, 4.2, 6.7, 9.5, 12.6, 16.1, 20, 24.4, 29.3, 34.7, 40.6, 47, 54.1, 61.8, 70.2,
    79.4, 89.5, 100.5, 112.5, 125.7, 140.1, 155.8, 172.9, 191.6, 212, 234.3, 258.6,
    285.1, 314, 345.5, 379.8, 417.1, 457.7, 500.8, 546.1, 592.9, 641.3, 691.3, 743,
    796.4, 851.6, 908.6, 967.5, 1028.3, 1091.2, 1157.8, 1230.4, 1311, 1401.6, 1501.8,
    1611.8, 1732.1, 1860.8, 1998.2, 2144.6, 2300.3, 2465.5, 2640.3, 2824.7, 3018.8,
    3222.4, 3435.3, 3657.2, 3887.7, 4126.2, 4372.1, 4624.5, 4882.6, 5145.4, 5411.8,
    5680.7, 5950.8
])

# Add depth_top and depth_bottom as variables associated with oceanModelLayer
ds = ds.assign_coords({
    "depth_top": ("oceanModelLayer", depth_top),
    "depth_bottom": ("oceanModelLayer", depth_bottom),
})


# Display the updated dataset
ds


<xarray.Dataset> Size: 4GB
Dimensions:          (time: 1, step: 1, oceanModelLayer: 72, cell_id: 12582912)
Coordinates:
  * time             (time) datetime64[ns] 8B 2024-11-25
  * step             (step) timedelta64[ns] 8B 1 days
  * oceanModelLayer  (oceanModelLayer) float64 576B 1.0 2.0 3.0 ... 71.0 72.0
    latitude         (cell_id) float64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
    longitude        (cell_id) float64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
    valid_time       (time, step) datetime64[ns] 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
    cell_ids         (cell_id) int64 101MB dask.array<chunksize=(65536,), meta=np.ndarray>
Dimensions without coordinates: cell_id
Data variables:
    avg_thetao       (time, step, oceanModelLayer, cell_id) float32 4GB dask.array<chunksize=(1, 1, 72, 65536), meta=np.ndarray>
    depth_top        (oceanModelLayer) float64 576B 0.0 2.0 ... 5.681e+03
    depth_bottom     (oceanModelLayer) float64 576B 2.0 4.2 ... 5.951e+03
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          1003
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-25T14:37 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
# Create the indices
indices_1 = np.arange(0 * 4**level, 1 * 4**level)
indices_2 = np.arange(3 * 4**level, 4 * 4**level)

# Combine the indices
combined_indices = np.concatenate([indices_1, indices_2])
combined_indices

array([      0,       1,       2, ..., 4194301, 4194302, 4194303])

In [9]:
ds=ds.sel(cell_id=combined_indices)

In [10]:
ds.to_zarr('ICON_0_3.zarr',mode='w')

In [9]:
ds=xr.open_zarr('ICON_0_3.zarr')
ds

<xarray.Dataset> Size: 654MB
Dimensions:          (time: 1, step: 1, oceanModelLayer: 72, cell_id: 2097152)
Coordinates:
    cell_ids         (cell_id) int64 17MB dask.array<chunksize=(65536,), meta=np.ndarray>
    depth_bottom     (oceanModelLayer) float64 576B dask.array<chunksize=(72,), meta=np.ndarray>
    depth_top        (oceanModelLayer) float64 576B dask.array<chunksize=(72,), meta=np.ndarray>
    latitude         (cell_id) float64 17MB dask.array<chunksize=(65536,), meta=np.ndarray>
    longitude        (cell_id) float64 17MB dask.array<chunksize=(65536,), meta=np.ndarray>
  * oceanModelLayer  (oceanModelLayer) float64 576B 1.0 2.0 3.0 ... 71.0 72.0
  * step             (step) timedelta64[ns] 8B 1 days
  * time             (time) datetime64[ns] 8B 2024-11-25
    valid_time       (time, step) datetime64[ns] 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
Dimensions without coordinates: cell_id
Data variables:
    avg_thetao       (time, step, oceanModelLayer, cell_id) float32 604MB dask.array<chunksize=(1, 1, 72, 65536), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          1003
    history:                 2024-11-25T14:37 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [12]:
test_plot = (
    ds["avg_thetao"]
    .isel(oceanModelLayer=0, step=0, time=0)
    .pint.quantify()
    .pint.to({"avg_thetao": "degC"})
    .pint.dequantify()
    .pipe(xdggs.decode)
    .compute()
)

In [13]:
test_plot.dggs.explore(center=0, cmap="viridis", alpha=0.5)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7fa1c3e0…

In [14]:
#bbox = {"latitude": [46, 51], "longitude": [-8, -1]}

lat = np.arange(46, 51, 0.01)
lon = np.arange(360-8, 360-1, 0.01)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))

(
 test_plot.dggs.sel_latlon(longitude=full_lon, latitude=full_lat)
).dggs.explore(center=0, cmap="viridis", alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7fa1c3e0…